In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate
from sklearn.gaussian_process import GaussianProcess

In [2]:
def make_a_model(pairNo, x, X, y, dy):
# Instanciate a Gaussian Process model
# Instanciate a Gaussian Process model
    gp = GaussianProcess(corr='squared_exponential', #?
                         regr = "quadratic", #?
                         theta0 = 1e-3,
                         thetaL = 1e-3,
                         thetaU = 1,
                         nugget = (dy / y) ** 2, #?
                         random_start=100)
# Fit to data using Maximum Likelihood Estimation of the parameters
    gp.fit(X, y)

# Make the prediction on the meshed x-axis (ask for MSE as well)
    y_pred, MSE = gp.predict(x, eval_MSE=True)
    sigma = np.sqrt(MSE)
    return y_pred, sigma

In [3]:
def plot_data(pairNo, dt, x, y_pred, sigma, X, y, dy, ob):
# Plot the function, the prediction and the 95% confidence interval based on
# the MSE
    plt.errorbar(X.ravel() + dt, y, dy, fmt='.', markersize=10, label='Observations ' + ob)

In [4]:
def plot_model(pairNo, dt, x, y_pred, sigma, X, y, dy, ob):
# Plot the function, the prediction and the 95% confidence interval based on
# the MSE
    plt.plot(x, y_pred, '-', label='Prediction ' + ob)
    plt.fill(np.concatenate([x, x[::-1]]),
        np.concatenate([y_pred - 1.9600 * sigma,
                       (y_pred + 1.9600 * sigma)[::-1]]),
        alpha=.5, ec='None', label='95% confidence interval ' + ob)

In [5]:
def cross_correlate_models(model1, model2, mode='same'):
    corr = correlate(model1, model2, mode=mode)
    abscorr = np.abs(corr)
    maxcorr = np.max(abscorr)
    return np.where(abscorr == maxcorr)[0], corr

In [6]:
dt_trues = [-57.53, -33.49, -108.85, 71.03, -33.82, -62.04, -61.6, -73.6, 41.9, 104.22]
periods = []
dt_preds = []
#dt_trues = [41.9, 104.22]
for pairNo, dt_true in zip(range(1, 11), dt_trues):
    lightcurve = np.loadtxt("/home/saas9842/PhD/Courses/AstroML/Project/tdc1/rung3/tdc1_rung3_double_pair%d.txt"%(pairNo),
                            skiprows=6, dtype={"names":("time", "lcA", "errA", "lcB", "errB"), 
                                               "formats":("f4", "f4", "f4", "f4", "f4")})
    x = np.atleast_2d(np.linspace(0, np.max(lightcurve['time']), 5000)).T

    XA = lightcurve['time'].T
    XA = XA.reshape((len(XA), 1))
#    yA = lightcurve['lcA']
    yA = (lightcurve['lcA'] - np.mean(lightcurve['lcA'])) / np.std(lightcurve['lcA'])
    dyA = lightcurve['errA']
    y_predA, sigmaA = make_a_model(pairNo, x, XA, yA, dyA) #A

    
#    XB = (lightcurve['time'] + dt_true).T
    XB = (lightcurve['time']).T
    XB = XB.reshape((len(XB), 1))
#    yB = lightcurve['lcB']
    yB = (lightcurve['lcB'] - np.mean(lightcurve['lcB'])) / np.std(lightcurve['lcB'])
    dyB = lightcurve['errB']
    y_predB, sigmaB = make_a_model(pairNo, x, XB, yB, dyB) #B
    

# Cross correlate the two models
    ind_maxcorr, corr = cross_correlate_models(y_predA, y_predB)
    t_maxcorr = x[ind_maxcorr]
    t_maxcorr = t_maxcorr[0][0]

# Autcorelate each of the models for an estimate of their periods individually
    ind_max_acfA, corrA = cross_correlate_models(y_predA, y_predA)
    t_max_acfA = x[ind_max_acfA]
    t_max_acfA = t_max_acfA[0][0]

    ind_max_acfB, corrB = cross_correlate_models(y_predB, y_predB)
    t_max_acfB = x[ind_max_acfB]
    t_max_acfB = t_max_acfB[0][0]

    periods.append(t_max_acfB)
    dt_preds.append(t_maxcorr)
    
    print "Pair " + str(pairNo) + " done!"

Pair 1 done!
Pair 2 done!
Pair 3 done!
Pair 4 done!
Pair 5 done!
Pair 6 done!
Pair 7 done!
Pair 8 done!
Pair 9 done!
Pair 10 done!


In [12]:
periods = np.array(periods)
dt_trues = np.array(dt_trues)
dt_preds = np.array(dt_preds)

plt.figure()
plt.scatter(dt_trues, periods, marker='o', c=periods, edgecolor='None', s=150)
plt.plot(dt_trues, dt_trues, linewidth=3)
plt.show()

plt.figure()
plt.scatter(dt_trues, periods, marker='o', c=dt_preds, edgecolor='None', s=150)
plt.plot(dt_trues, dt_trues, linewidth=3)
plt.show()

In [15]:
# Plot everything
    plt.figure()

    plot_data(pairNo, 0, x, y_predA, sigmaA, XA, yA, dyA, "A") #A
    plot_model(pairNo, 0, x, y_predA, sigmaA, XA, yA, dyA, "A - period: " + str(t_max_acfA)) #A
    
    plot_data(pairNo, 0, x, y_predB, sigmaB, XB, yB, dyB, "B") #B
    plot_model(pairNo, 0, x, y_predB, sigmaB, XB, yB, dyB, "B - period: " + str(t_max_acfB)) #B
    
    plt.plot(x, (corr - np.mean(corr)) / np.std(corr),
             '-y', linewidth=3, label="Cross correlation")
    plt.xlabel('t [days]')
    plt.ylabel('normalized flux [arbitrary]')
#    plt.ylim(-2.0, 2.5)
    plt.title('PairNo: ' + str(pairNo) +
              ' - true time delay: ' + str(dt_true) +
              ' - maximum correlation at: ' + str(t_maxcorr))
    
    plt.legend().draggable()
    plt.show()

104.22 42.3452325231
